In [30]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl
import torch

from rxitect.models.pchembl_val_predictor import PChEMBLValueRegressor
from rxitect.data.datasets import SelfiesQsarDataset, SelfiesQsarDataModule

In [2]:
df = pd.read_parquet("../data/processed/CHEMBL240_dataset.pq")

In [4]:
df = df.dropna(subset=['pchembl value'])

In [23]:
reg = PChEMBLValueRegressor(lr=0.05)

In [29]:
dm = SelfiesQsarDataModule()
trainer = pl.Trainer(accelerator="gpu", fast_dev_run=True)

MisconfigurationException: GPUAccelerator can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

In [31]:
torch.cuda.is_available()

False